## Objetivo
El objetivo del análisis es crear un modelo de regresión que pueda predecir la calificación de una película o serie según el título, año de creación, duración, actores principales y descripción.

En primer lugar, importamos las bibliotecas que vamos a usar

In [15]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Análisis de los datos
Vamos a usar el dataset IMDB, pero primero tenemos que analizar qué datos tiene y cuáles nos sirven.

In [16]:
dataset = pd.read_csv('/home/solmoon/aprendizaje_automatico/IMBD.csv')
df = pd.DataFrame(dataset)
print('Primeras filas:')
df.head()

Primeras filas:


,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,Decades after their 1984 All Valley Karate Tou...,"['Ralph Macchio, ', 'William Zabka, ', 'Courtn...","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of...,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda St...","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer...,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan...","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,When a psychiatrist shelters a mysterious cult...,"['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo...","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,A Street Kid trying to survive in a technology...,"['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi ...","15,413"


In [17]:
print('Las columnas son :\n')
print(f'{df.columns}\n')

print(f'La forma completa del dataset es: {df.shape}')

Las columnas son :

Index(['title', 'year', 'certificate', 'duration', 'genre', 'rating',
       'description', 'stars', 'votes'],
      dtype='object')

La forma completa del dataset es: (9957, 9)


Vemos que tenemos las siguientes columnas:
- **Título** de la película o serie
- **Año** de estreno
- **Certificado**
- **Géneros**
- **Rating**
- **Descripción**
- **Actores principales**
- **Votos**

Analizamos qué datos tenemos en cada columna, y cuáles pueden llegar a ser nulos o vacíos.

In [18]:
df.describe(include='all')

,title,year,certificate,duration,genre,rating,description,stars,votes
count,9957,9430,6504,7921,9884,8784.000000,9957,9957,8784
unique,7912,498,20,291,569,NaN,9433,8615,4862
top,Top Gear,(2020),TV-MA,60 min,Comedy,NaN,Add a Plot,[],30
freq,92,663,2520,385,859,NaN,434,403,33
mean,NaN,NaN,NaN,NaN,NaN,6.764515,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,1.214840,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,1.700000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,6.100000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,6.900000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,7.600000,NaN,NaN,NaN


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


Vemos que hay varias columnas que tienen muchos datos nulos, particularmente certificado, duración, rating y votos. Como para el modelo que queremos armar certificado y votos no son datos que van a ayudar, los quitamos del dataframe. 
Luego, como rating y descripción son los datos que más nos van a ayudar, quitamos las filas donde esas columnas tienen datos nulos. Si bien perdemos algunas filas, nos va a ayudar a entrenar mejor al modelo.

In [20]:
df = df.drop(columns=['certificate', 'votes'])
df = df.dropna(subset=['rating', 'description'])

In [21]:
print('El nuevo dataset tiene:\n')
print(f'Columnas: {df.columns}\n')

print(f'Filas y columnas: {df.shape}')

El nuevo dataset tiene:

Columnas: Index(['title', 'year', 'duration', 'genre', 'rating', 'description', 'stars'], dtype='object')

Filas y columnas: (8784, 7)
